In [ ]:
!pip install pyspark py4j

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
from google.colab import files
files.upload()

Saving Student_Grades_Data.csv to Student_Grades_Data.csv


{'Student_Grades_Data.csv': b'Time_to_Study,Grades\n1,1.5\n5,2.7\n7,3.1\n3,2.1\n2,1.8\n9,3.9\n6,2.9\n12,4.5\n11,4.3\n2,1.8\n4,2.4\n8,3.5\n13,4.8\n9,3.9\n14,5\n10,4.1\n6,2.9\n12,4.5\n1,1.5\n4,2.4\n14,5\n10,4.1\n11,4.3\n4,2.4\n5,2.7\n8,3.5\n1,1.5\n2,1.8\n3,2.1\n7,3.1\n8,3.5\n14,5\n7,3.1\n8,3.5\n1,1.5\n2,1.8\n3,2.1\n4,2.4\n5,2.7\n6,2.9\n7,3.1\n8,3.5\n9,3.9\n10,4.1\n11,4.3\n12,4.5\n13,4.8\n14,5\n8,3.5\n2,1.8\n'}

In [ ]:
data = spark.read.csv('Student_Grades_Data.csv', header=True, inferSchema=True)

In [ ]:
data.printSchema()

root
 |-- Time_to_Study: integer (nullable = true)
 |-- Grades: double (nullable = true)



In [ ]:
data.show()

+-------------+------+
|Time_to_Study|Grades|
+-------------+------+
|            1|   1.5|
|            5|   2.7|
|            7|   3.1|
|            3|   2.1|
|            2|   1.8|
|            9|   3.9|
|            6|   2.9|
|           12|   4.5|
|           11|   4.3|
|            2|   1.8|
|            4|   2.4|
|            8|   3.5|
|           13|   4.8|
|            9|   3.9|
|           14|   5.0|
|           10|   4.1|
|            6|   2.9|
|           12|   4.5|
|            1|   1.5|
|            4|   2.4|
+-------------+------+
only showing top 20 rows



In [ ]:
feature_cols = data.columns[:-1] 
from pyspark.ml.feature import VectorAssembler
vect_assembler = VectorAssembler(inputCols=feature_cols,outputCol="features")

In [ ]:
data_w_features = vect_assembler.transform(data)

In [ ]:
data_w_features.show()

+-------------+------+--------+
|Time_to_Study|Grades|features|
+-------------+------+--------+
|            1|   1.5|   [1.0]|
|            5|   2.7|   [5.0]|
|            7|   3.1|   [7.0]|
|            3|   2.1|   [3.0]|
|            2|   1.8|   [2.0]|
|            9|   3.9|   [9.0]|
|            6|   2.9|   [6.0]|
|           12|   4.5|  [12.0]|
|           11|   4.3|  [11.0]|
|            2|   1.8|   [2.0]|
|            4|   2.4|   [4.0]|
|            8|   3.5|   [8.0]|
|           13|   4.8|  [13.0]|
|            9|   3.9|   [9.0]|
|           14|   5.0|  [14.0]|
|           10|   4.1|  [10.0]|
|            6|   2.9|   [6.0]|
|           12|   4.5|  [12.0]|
|            1|   1.5|   [1.0]|
|            4|   2.4|   [4.0]|
+-------------+------+--------+
only showing top 20 rows



In [ ]:
finalized_data = data_w_features.select("features","Grades")

finalized_data.show()

+--------+------+
|features|Grades|
+--------+------+
|   [1.0]|   1.5|
|   [5.0]|   2.7|
|   [7.0]|   3.1|
|   [3.0]|   2.1|
|   [2.0]|   1.8|
|   [9.0]|   3.9|
|   [6.0]|   2.9|
|  [12.0]|   4.5|
|  [11.0]|   4.3|
|   [2.0]|   1.8|
|   [4.0]|   2.4|
|   [8.0]|   3.5|
|  [13.0]|   4.8|
|   [9.0]|   3.9|
|  [14.0]|   5.0|
|  [10.0]|   4.1|
|   [6.0]|   2.9|
|  [12.0]|   4.5|
|   [1.0]|   1.5|
|   [4.0]|   2.4|
+--------+------+
only showing top 20 rows



In [ ]:
train_dataset, test_dataset = finalized_data.randomSplit([0.7, 0.3])

In [ ]:
train_dataset.describe().show()

+-------+------------------+
|summary|            Grades|
+-------+------------------+
|  count|                37|
|   mean| 3.262162162162162|
| stddev|1.1206028514825188|
|    min|               1.5|
|    max|               5.0|
+-------+------------------+



In [ ]:
test_dataset.describe().show()

+-------+------------------+
|summary|            Grades|
+-------+------------------+
|  count|                13|
|   mean|3.1076923076923078|
| stddev| 1.094274141810861|
|    min|               1.5|
|    max|               4.5|
+-------+------------------+



In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
LinReg = LinearRegression(featuresCol="features", labelCol="Grades")

In [ ]:
model = LinReg.fit(train_dataset)

In [ ]:
pred = model.evaluate(test_dataset)

In [ ]:
pred.predictions.show()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+--------+------+------------------+
|features|Grades|        prediction|
+--------+------+------------------+
|   [1.0]|   1.5|1.5615943297164854|
|   [1.0]|   1.5|1.5615943297164854|
|   [2.0]|   1.8|1.8316415820791034|
|   [3.0]|   2.1|2.1016888344417217|
|   [5.0]|   2.7|2.6417833391669583|
|   [6.0]|   2.9|2.9118305915295766|
|   [8.0]|   3.5| 3.451925096254813|
|   [8.0]|   3.5| 3.451925096254813|
|   [9.0]|   3.9| 3.721972348617431|
|  [10.0]|   4.1|3.9920196009800497|
|  [10.0]|   4.1|3.9920196009800497|
|  [11.0]|   4.3|4.2620668533426675|
|  [12.0]|   4.5| 4.532114105705285|
+--------+------+------------------+



In [ ]:
coefficient = model.coefficients
print ("The coefficient of the model is : %a" %coefficient)

The coefficient of the model is : DenseVector([0.27])


In [ ]:
intercept = model.intercept
print ("The Intercept of the model is : %f" %intercept)

The Intercept of the model is : 1.291547


In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluation = RegressionEvaluator(labelCol="Grades", predictionCol="prediction")

# Root Mean Square Error
rmse = evaluation.evaluate(pred.predictions, {evaluation.metricName: "rmse"})
print("RMSE: %.3f" % rmse)

# Mean Square Error
mse = evaluation.evaluate(pred.predictions, {evaluation.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = evaluation.evaluate(pred.predictions, {evaluation.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = evaluation.evaluate(pred.predictions, {evaluation.metricName: "r2"})
print("r2: %.3f" %r2)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


RMSE: 0.076
MSE: 0.006
MAE: 0.061
r2: 0.995


In [ ]:
unlabeled_dataset = test_dataset.select('features')

In [ ]:
unlabeled_dataset.show()

+--------+
|features|
+--------+
|   [1.0]|
|   [1.0]|
|   [2.0]|
|   [3.0]|
|   [5.0]|
|   [6.0]|
|   [8.0]|
|   [8.0]|
|   [9.0]|
|  [10.0]|
|  [10.0]|
|  [11.0]|
|  [12.0]|
+--------+



In [ ]:
new_predictions = model.transform(unlabeled_dataset)

In [ ]:
new_predictions.show()

+--------+------------------+
|features|        prediction|
+--------+------------------+
|   [1.0]|1.5615943297164854|
|   [1.0]|1.5615943297164854|
|   [2.0]|1.8316415820791034|
|   [3.0]|2.1016888344417217|
|   [5.0]|2.6417833391669583|
|   [6.0]|2.9118305915295766|
|   [8.0]| 3.451925096254813|
|   [8.0]| 3.451925096254813|
|   [9.0]| 3.721972348617431|
|  [10.0]|3.9920196009800497|
|  [10.0]|3.9920196009800497|
|  [11.0]|4.2620668533426675|
|  [12.0]| 4.532114105705285|
+--------+------------------+

